# Example napari-convpaint in jupyter notebooks
Although the implementation of different plugins in napari promotes its use as a standalone software for image segmentation and analysis, Napari is in the first place an image viewer for the python ecosystem.

In this example, we will demonstrate how to use napari-convpaint within a jupyter notebook. This approach is very valuable for building reproducible, well-documented, and sharable analysis workflows.

## Installation
We recommend creating a new environment and installing napari and napari-convpaint as described [in the documentation](https://guiwitz.github.io/napari-convpaint/book/Installation.html).

In addition to the installation of napari-convpaint, make sure that in the same environment, you have also jupyter lab or jupyter notebook. Follow the instructions [here](https://jupyter.org/install).

## Import the necessary packages

In [1]:
import napari
from skimage import data

## Import images
For this example we have chosen an image that is available on scikitimage, and it is also included in the napari builtins image samples.

In [2]:
nuclei = data.human_mitosis()

In [3]:
nuclei.shape

(512, 512)

In [4]:
nuclei.dtype

dtype('uint8')

## Create a napari viewer

Create a napari viewer and call it viewer

In [5]:
viewer = napari.Viewer()

Here is how to add the widget in napari direcly from the jupyter notebook

In [6]:
from napari_convpaint.conv_paint import ConvPaintWidget
widget = ConvPaintWidget(napari_viewer=viewer)
viewer.window.add_dock_widget(widget)

In [7]:
viewer.add_image(nuclei, name='nuc')

<Image layer 'nuc' at 0x2aecd72b0>

# Pixel classification using napari-convpaint
Now that you have added the image to the viewer, use napari-convpaint to generate the 'prediction' layer. 

The prediction assigns each pixel to the most probable class to which the pixel belongs based on the trained model. In fact, it is an array of the size of the image with the class value for each pixel. Usually, for segmentation tasks we use two classes 'background'(label=1 in napari) and 'foreground'(label=2), but convpaint can predict many more classes.

Once we have obtained the prediction layer we can import the data in jupyter and continue from here to obtain the object segmentation using the [scikit-image function 'regionprops'](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_regionprops.html)

In [9]:
import skimage.measure

In [10]:
prediction = widget.viewer.layers['prediction'].data

As we have labeled the images with label1 being the background and label2 being the foreground, if we instead want a threshold image with background = 0 and foreground = 1 we can simply subtract 1 from each element of the array. If we skip this passage the first big object will be the background.

In [11]:
binary = prediction == 2

In [12]:
binary

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True]])

## Label image

In [13]:
labeled_image, count = skimage.measure.label(binary, return_num=True)

In [14]:
count

280

To view the result of the labeling we can now open the labeled image in napari

In [15]:
viewer.add_labels(labeled_image, name='nuc_labels')

<Labels layer 'nuc_labels' at 0x2b1460760>

## Watershed to split touching nuclei
Here we could use the [watershed function](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html) of scikit-image

In [16]:
import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max

In [17]:
distance = ndi.distance_transform_edt(labeled_image)
coords = peak_local_max(distance, min_distance=5, footprint=np.ones((3, 3)), labels=labeled_image)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels_split = watershed(-distance, markers, mask=labeled_image)

In [18]:
viewer.add_labels(labels_split, name='watershed')

<Labels layer 'watershed' at 0x30a252e00>

## Filter out small objects

First we need to measure the area of the objects after watershed

In [46]:
from skimage.measure import regionprops_table
from skimage.util import map_array
import pandas as pd

In [28]:
labels_data = regionprops_table(labels_split,
                               properties=('label','area'))

In [31]:
labels_table = pd.DataFrame(labels_data)

In [33]:
labels_table

,label,area
0,1,64.0
1,2,74.0
2,3,129.0
3,4,113.0
4,5,79.0
...,...,...
336,337,104.0
337,338,76.0
338,339,48.0
339,340,54.0


In [34]:
area_thr = 50

In [36]:
area_labels = labels_table['label']* (labels_table['area']>area_thr)

In [47]:
filtered_labels = map_array(labels_split,
                           np.asarray(labels_table['label']),
                           np.asarray(area_labels))

In [48]:
filtered_labels, count_filter = skimage.measure.label(filtered_labels, return_num=True)

In [49]:
viewer.add_labels(filtered_labels, name='watershed')

<Labels layer 'watershed [1]' at 0x30bbf4070>